In [1]:
import csv
import itertools
import os
import re
import unicodedata
from collections import namedtuple
from glob import glob
import requests
import os
import re
import pandas as pd
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import numpy as np
import MDAExtract

In [2]:
dfFin=pd.read_csv('/Volumes/Elements/MasterThesis/MAstertheisMertut/DataFiles/data2.2imp.new.1.csv', sep=',')
dfFin.shape

(4090, 31)

In [3]:
dfFin=dfFin.rename(columns={"Company.Name": "CONAME","time":"FYEAR"})
dfFin["CONAME"]=dfFin["CONAME"]+" "+ dfFin["FYEAR"].map(str)
dfFin.head()

,CONAME,FYEAR,AAER.Dummy,AccRec.DbtsT,Sales.TurnNt,COGS.costsgd,Assets.Total,CurrAssetTot,PPE.TotalNet,DeprAmortTot,...,GrProfitMarg,EBIT.Marginn,NtProfitMarg,CashFLMargin,ROA.finratio,ROE.RnCommEQ,EBITDAMargin,Acc.Payables,InventoryPrx,Primary.SIC.Code
0,3COM CORP 2001,2001,0,147.113,1477.932,843.553,2526.792,1663.044,676.154,266.820,...,42.923,-25.662,-40.323,-22.270,-23.585,-30.558,-7.608,125.903,61.777,3576
1,3COM CORP 2002,2002,0,90.290,932.866,375.604,2062.360,1653.180,248.790,162.523,...,59.737,-5.957,-24.665,-7.243,-11.157,-13.388,11.121,105.583,27.068,3576
2,3COM CORP 2003,2003,0,66.372,698.884,351.125,1820.818,1519.677,72.452,111.714,...,49.759,-25.483,-49.631,-33.646,-19.050,-23.138,-9.829,80.408,27.679,3576
3,3COM CORP 2004,2004,0,61.664,651.244,372.649,1592.967,977.509,69.535,53.256,...,42.779,-26.530,-30.048,-21.870,-12.284,-15.349,-18.568,99.632,29.311,3576
4,3COM CORP 2005,2005,0,178.344,794.807,442.961,1861.361,1249.345,89.109,44.685,...,44.268,-16.764,-12.667,-7.044,-5.409,-8.373,-11.141,153.245,148.819,3576


In [4]:
dfFin.shape

(4090, 31)

In [5]:
df2=pd.read_csv('/Volumes/Elements/MasterThesis/Thesis/DataSheets/index.10k.csv',header=None, sep=',')
df2.columns=["FORM","CONAME","CIK","FDATE","SECFNAME"]
df2["FYEAR"]=df2.apply(lambda row: str(row.FDATE[:4]), axis=1)
df2["CONAME"]=df2["CONAME"]+" "+ df2["FYEAR"].map(str)
df2.head(){{}}

,FORM,CONAME,CIK,FDATE,SECFNAME,FYEAR
0,10-K,AMERICAN MEDICAL HOLDINGS INC 1993,861439,1993-11-29,edgar/data/861439/0000912057-94-000263.txt,1993
1,10-K,DISNEY WALT CO 1993,29082,1993-12-22,edgar/data/29082/0000950131-94-000021.txt,1993
2,10-K,ELIZABETHTOWN GAS CO 1993,32377,1993-12-13,edgar/data/32377/0000032377-94-000001.txt,1993
3,10-K,INTERNATIONAL GAME TECHNOLOGY 1993,353944,1993-12-23,edgar/data/353944/0000353944-94-000005.txt,1993
4,10-K,ABBOTT LABORATORIES 1994,1800,1994-03-03,edgar/data/1800/0000912057-94-000771.txt,1994


In [6]:
ListConame=dfFin.CONAME.tolist()
ListConame
ListConame2=df2.CONAME.tolist()

In [7]:
df2subset=df2[df2["CONAME"].isin(ListConame)]

In [8]:
#this is the whole subset of all 10kindex csv for which Meirut has Compustat Data!!! Lets see how many are included
#in the Fraud Data Set and exclude those to retrieve all NON Fraud Cases for which we have COmpustat Data from Meirut
df2subset.head()

,FORM,CONAME,CIK,FDATE,SECFNAME,FYEAR
15153,10-K,3DFX INTERACTIVE INC 1998,1010026,1998-02-18,edgar/data/1010026/0000891618-98-000818.txt,1998
15159,10-K,AAON INC 1998,824142,1998-03-26,edgar/data/824142/0001026608-98-000005.txt,1998
15180,10-K,ACTEL CORP 1998,907687,1998-03-31,edgar/data/907687/0000907687-98-000026.txt,1998
15182,10-K,ACUSON CORP 1998,717014,1998-03-27,edgar/data/717014/0001012870-98-000763.txt,1998
15187,10-K,ADC TELECOMMUNICATIONS INC 1998,61478,1998-01-16,edgar/data/61478/0001047469-98-001384.txt,1998


In [9]:
df2subset.shape

(3952, 6)

In [10]:
df2subset["CONAME"]=df2subset.apply(lambda row: row.CONAME[:-5], axis=1)

/Users/patricia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df2subset.head()

,FORM,CONAME,CIK,FDATE,SECFNAME,FYEAR
15153,10-K,3DFX INTERACTIVE INC,1010026,1998-02-18,edgar/data/1010026/0000891618-98-000818.txt,1998
15159,10-K,AAON INC,824142,1998-03-26,edgar/data/824142/0001026608-98-000005.txt,1998
15180,10-K,ACTEL CORP,907687,1998-03-31,edgar/data/907687/0000907687-98-000026.txt,1998
15182,10-K,ACUSON CORP,717014,1998-03-27,edgar/data/717014/0001012870-98-000763.txt,1998
15187,10-K,ADC TELECOMMUNICATIONS INC,61478,1998-01-16,edgar/data/61478/0001047469-98-001384.txt,1998


In [13]:
dfAllFraudCases=pd.read_csv('/Volumes/crajapax/DataCollection/Datasheets/AllFraudCases.csv', sep=',')
dfAllFraudCases.head()

,FORM,CONAME,CIK,FDATE,SECFNAME,target
0,10-K,POLICY MANAGEMENT SYSTEMS CORP,356226,4/13/94,edgar/data/356226/0000356226-94-000007.txt,1
1,10-K,STRUCTURAL DYNAMICS RESEARCH CORP /OH/,820235,3/28/95,edgar/data/820235/0000906318-95-000015.txt,1
2,10-K,CUC INTERNATIONAL INC /DE/,723612,4/25/96,edgar/data/723612/0000950112-96-001227.txt,1
3,10-K,HEALTHSOUTH CORP,785161,3/27/97,edgar/data/785161/0001005150-97-000195.txt,1
4,10-K,CUC INTERNATIONAL INC /DE/,723612,5/1/97,edgar/data/723612/0001005477-97-001218.txt,1


In [14]:
ListSECFNAME=df2subset.SECFNAME.tolist()
ListSECFNAME2=dfAllFraudCases.SECFNAME.tolist()
ListSECFNAME3=list(set(ListSECFNAME)-set(ListSECFNAME2))

In [15]:
dfFraudSubset=df2subset[df2subset["SECFNAME"].isin(ListSECFNAME2)]

In [16]:
dfFraudSubset.head()

,FORM,CONAME,CIK,FDATE,SECFNAME,FYEAR
15764,10-K,CAMBREX CORP,820081,1998-03-20,edgar/data/820081/0000950123-98-002731.txt,1998
21383,10-K,TRIDENT MICROSYSTEMS INC,859475,1998-09-25,edgar/data/859475/0000891618-98-004302.txt,1998
22337,10-K,ANIKA THERAPEUTICS INC,898437,1999-03-31,edgar/data/898437/0001047469-99-013073.txt,1999
22441,10-K,AURORA FOODS INC,1033523,1999-03-12,edgar/data/1033523/0000929624-99-000423.txt,1999
22692,10-K,CAMBREX CORP,820081,1999-03-22,edgar/data/820081/0000950123-99-002355.txt,1999


In [17]:
#181 ist die Anzahl der Fraud Cass für die wir auch Compustat Daten besitzen
dfFraudSubset.shape

(181, 6)

In [18]:
dfNOFraudSubset=df2subset[df2subset["SECFNAME"].isin(ListSECFNAME3)]

In [19]:
dfNOFraudSubset.shape

(3771, 6)

In [20]:
dfNOFraudSubset["target"]=0

/Users/patricia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
dfNOFraudSubset.head()

,FORM,CONAME,CIK,FDATE,SECFNAME,FYEAR,target
15153,10-K,3DFX INTERACTIVE INC,1010026,1998-02-18,edgar/data/1010026/0000891618-98-000818.txt,1998,0
15159,10-K,AAON INC,824142,1998-03-26,edgar/data/824142/0001026608-98-000005.txt,1998,0
15180,10-K,ACTEL CORP,907687,1998-03-31,edgar/data/907687/0000907687-98-000026.txt,1998,0
15182,10-K,ACUSON CORP,717014,1998-03-27,edgar/data/717014/0001012870-98-000763.txt,1998,0
15187,10-K,ADC TELECOMMUNICATIONS INC,61478,1998-01-16,edgar/data/61478/0001047469-98-001384.txt,1998,0


In [22]:
#extract No Fraud MDAs
dfNOFraudSubset.to_csv('NoFraudCases.csv', header=True, index=False)

In [25]:
filepath='/Volumes/crajapax/DataCollection/NoFraudCases.csv'
name="downloadlistNoFraudMda"
#download="/Volumes/Elements/MasterThesis/Master/Data/mda_nofraud/downloadlistNoFraudMda.txt"

In [26]:
MdaExtract=MDAExtract.downloadMDA(filepath,name)

/Volumes/Elements/MasterThesis/Master/Data/mda/downloadlistNoFraudMda2.txt
1121788_0001144204-10-009642
1086319_0000950123-10-020605
1131324_0000950123-10-024793
355811_0000950123-10-015388
41023_0001193125-10-055455
746210_0001415889-10-000061
943861_0001193125-10-042262
851310_0000950123-10-019274
879169_0001047469-10-001834
1138639_0001193125-10-044817
1051114_0001188112-10-000853
1104506_0001104506-10-000004
1040416_0001193125-10-057177
1145197_0000950123-10-022584
1087432_0001193125-10-055752
NO MD&A
736012_0001104659-10-016982
1111928_0000950123-10-024694
1106935_0000950123-10-025259
1114714_0001144204-10-012295
1096788_0001193125-10-056792
855658_0001193125-10-052435
1033905_0000950123-10-016925
773141_0001193125-10-022961
910329_0001144204-10-016775
701811_0001193125-10-059520
856982_0001104659-10-013524
932111_0000932111-10-000010
1050446_0001193125-10-038870
911109_0000950123-10-019227
1085770_0000950123-10-021303
1265888_0000950123-10-018171
71691_0001193125-10-036478
106533

1040593_0001193125-12-089501
1020214_0001193125-12-096720
20212_0001193125-12-109920
1168054_0001047469-12-001276
30828_0001193125-12-118967
22701_0000897101-12-000451
23111_0001193125-12-078045
24104_0001104659-12-022780
1023131_0001047469-12-001714
1209821_0001047469-12-001741
1108205_0001193125-12-089082
885306_0001193125-12-101762
1005201_0001047469-12-002308
1042893_0001193125-12-078769
1082038_0001193125-12-094391
31347_0001193125-12-115264
867374_0001193125-12-070404
1244937_0001193125-12-117111
1027207_0001047469-12-002000
887936_0001193125-12-078356
1086319_0001104659-12-022013
1131324_0001047469-12-002637
355811_0001193125-12-071341
943861_0001193125-12-076931
1031203_0001193125-12-052522
1276591_0001193125-12-117396
1080099_0001080099-12-000006
1087432_0001193125-12-089195
1111928_0001193125-12-081509
56047_0001193125-12-083165
771266_0001193125-12-117581
1102993_0001144204-12-014539
63754_0001193125-12-026335
1099590_0001193125-12-085292
1179929_0001193125-12-089404
906709_

743238_0001437749-15-004162
95574_0001437749-15-003455
875657_0000875657-15-000010
1142701_0001047469-15-001433
755001_0001193125-15-023683
881524_0001047469-15-001202
878828_0000930413-15-001568
1130591_0001193125-15-069675
815094_0001193125-15-204302
890491_0001193125-15-237080
724571_0001144204-15-023306
1100270_0001206774-15-001119
1087423_0001193125-15-153023
1019671_0001193125-15-121530
923601_0001144204-15-039740
912615_0001193125-15-116065
1138723_0001047469-15-007028
2034_0001571049-15-007509
6284_0001193125-15-329227
19446_0001104659-15-068104
1039065_0001558370-15-001687
NO MD&A
943819_0001193125-15-281887
933034_0000950123-15-009734
1177648_0001193125-15-401602
1001115_0001564590-15-010914
1005731_0001213900-15-007660
50493_0000050493-15-000023
1057060_0001564590-15-011336
70793_0001047469-15-008748
924717_0001564590-15-011278
831641_0001047469-15-008864
910638_0000910638-16-000016
824142_0000824142-16-000118
1070494_0001193125-16-483533
926282_0001193125-16-475816
1098972_